# Relocation algorithm

Import the packages necessary to read the datasets from disk and set the base path of the datasets

In [12]:
import pathlib
import os
import pandas as pd

base_path = pathlib.Path(os.environ.get("SAMPLES_DIR", ""))

Read the input dataset containing all the data for the relocation problem

In [13]:
relocation_df = pd.read_csv(
    base_path / "relocation" / "PARABIAGO_unsat_epsg32632_wkt_AMELIA.csv",
    sep=';')

The input dataset consists of two merged datasets:

1. **Area Population Data**: contains information about geographical areas and their population counts

2. **Facilities Data**: contains locations of facilities (hospitals) and their available resources (beds, personnel)

These two datasets are merged into a single dataframe.

In [14]:
relocation_df[relocation_df['popolazion'].notna()].head()

,ID,popolazion,geometry,usable,beds,personnel
0,CUSANO MILANINO,189.12,"POLYGON ((515162.653766 5045374.60466, 515207....",NaN,NaN,NaN
1,ALBIZZATE,51.35,"POLYGON ((484862.09332 5065331.546162, 484862....",NaN,NaN,NaN
2,CESATE,143.09,"POLYGON ((504759.273301 5050484.89305, 504759....",NaN,NaN,NaN
3,CERRO MAGGIORE,150.31,"POLYGON ((495239.306843 5050834.835681, 495258...",NaN,NaN,NaN
4,CESANO MADERNO,395.49,"POLYGON ((508683.495302 5054098.191944, 508683...",NaN,NaN,NaN


In [18]:
relocation_df[relocation_df['personnel'].notna()].head()

,ID,popolazion,geometry,usable,beds,personnel
175,IRCCS IST. SCIENTIFICO DI RIABILITAZIONE - MI,NaN,POINT (520498.9153106844 5032321.101216923),1.0,80.0,16.0
176,POLICLINICO SAN MARCO - OSIO SOTTO,NaN,POINT (546112.3899832936 5050195.388898186),1.0,297.0,60.0
177,PRESIDIO OSPEDALIERO GARDONE V.T.,NaN,POINT (592159.6966155188 5058759.244051828),1.0,109.0,22.0
178,OSPEDALE CIVILE DI LEGNANO,NaN,POINT (491210.4726045889 5047456.290382859),1.0,481.0,97.0
179,IST. CLIN. HUMANITAS - ROZZANO,NaN,POINT (513109.4762980791 5024346.926455218),1.0,754.0,151.0


The relocation algorithm is executed on the dataset using two types of resources with specific requirements per demand unit:

* **Beds**: one bed is required to serve each unit of demand
* **Personnel**: 0.2 personnel units are needed per unit of demand (1 staff member can serve 5 demand units)

The algorithm will:
- Match demand (population) with available facility resources
- Consider both bed capacity and personnel constraints
- Optimize the allocation while minimizing transportation costs


In [16]:
from libadalina_core.sedona_utils import EPSGFormats
from libadalina_analytics.relocation.algorithms import relocation_algorithm
from libadalina_analytics.relocation.models import RelocationResource

relocation_solution = relocation_algorithm(
    relocation_df,
    epsg=EPSGFormats.EPSG32632,
    id_column='ID',
    demand_column='popolazion',
    resources=[
        RelocationResource(column_name='beds', amount=1.0),
        RelocationResource(column_name='personnel', amount=0.2)
    ]
)


Running HiGHS 1.11.0 (git hash: n/a): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 66055 rows; 97015 cols; 354955 nonzeros; 180 integer variables (180 binary)
Coefficient ranges:
  Matrix [1e+00, 4e+04]
  Cost   [6e+00, 2e+03]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+00]
Presolving model
895 rows, 96475 cols, 224275 nonzeros  0s
895 rows, 96475 cols, 224275 nonzeros  0s

Solving MIP model with:
   895 rows
   96475 cols (0 binary, 0 integer, 0 implied int., 96475 continuous, 0 domain fixed)
   224275 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       

The relocation algorithm outputs a new DataFrame containing:
* Unserved population for each area
* Transportation costs
* Cost penalties for unserved demand


In [17]:
relocation_solution.head()

,area,demand,geometry,unserved,unserved_fract,cost_transport,cost_unserved
0,CUSANO MILANINO,189.12,"POLYGON ((515162.653766 5045374.60466, 515207....",0.0,0.0,2610.289095,0.0
1,ALBIZZATE,51.35,"POLYGON ((484862.09332 5065331.546162, 484862....",0.0,0.0,435.942921,0.0
2,CESATE,143.09,"POLYGON ((504759.273301 5050484.89305, 504759....",0.0,0.0,5744.942451,0.0
3,CERRO MAGGIORE,150.31,"POLYGON ((495239.306843 5050834.835681, 495258...",0.0,0.0,1014.808312,0.0
4,CESANO MADERNO,395.49,"POLYGON ((508683.495302 5054098.191944, 508683...",0.0,0.0,12942.243358,0.0
